In [2]:
#!pip install auto-sklearn wandb

Random State for reproduceability

In [3]:
dagstuhl_seed=23372

# Data

In [4]:
import numpy as np
import pandas as pd

In [6]:
df = pd.read_csv('../../data/preprocessed.csv')
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df = df.replace(['inactive', 'active'], [0,1]) # swap -1 (inactive) and 0 (unknown)

df_train = pd.read_csv('../../data/preprocessed-train.csv')
df_train = df_train.loc[:, ~df_train.columns.str.contains('^Unnamed')]
df_train = df_train.replace(['inactive', 'active'], [0,1]) # swap -1 (inactive) and 0 (unknown)

df_test = pd.read_csv('../../data/preprocessed-test.csv')
df_test = df_test.loc[:, ~df_test.columns.str.contains('^Unnamed')]
df_test = df_test.replace(['inactive', 'active'], [0,1]) # swap -1 (inactive) and 0 (unknown)

df.tail()

,label,0,1,2,3,4,5,6,7,8,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
4306,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4307,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4308,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4309,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4310,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
y_train = df_train['label']
X_train = df_train.loc[:, df_train.columns != "label"]

y_test = df_train['label']
X_test = df_train.loc[:, df_train.columns != "label"]

X_train.to_numpy().shape

(3592, 1024)

# AutoML Libraries

In [8]:
time_min = 60
time_sec = time_min * 60

## Auto-Sklearn2

In [11]:
from autosklearn.experimental.askl2 import AutoSklearn2Classifier
from sklearn.metrics import accuracy_score

In [15]:
logging_config = {
    "version": 1,
    "disable_existing_loggers": True,
    "formatters": {
        "custom": {
            # More format options are available in the official
            # `documentation <https://docs.python.org/3/howto/logging-cookbook.html>`_
            "format": "%(asctime)s - %(name)s - %(levelname)s - %(message)s"
        }
    },
    # Any DEBUG level msg will be printed to the console
    "handlers": {
        "console": {
            "level": "DEBUG",
            "formatter": "custom",
            "class": "logging.StreamHandler",
            "stream": "ext://sys.stdout",
        },
    },
    "loggers": {
        "": {  # root logger
            "level": "DEBUG",
        },
        "Client-EnsembleBuilder": {
            "level": "DEBUG",
            "handlers": ["console"],
        },
    },
}

In [16]:
automl = AutoSklearn2Classifier(
    time_left_for_this_task=100,
    n_jobs=-1,
    delete_tmp_folder_after_terminate=False,
    logging_config=logging_config,
    tmp_folder="./autosk2-models",
)
automl.fit(X_train, y_train)

/opt/conda/lib/python3.9/site-packages/autosklearn/experimental/selector.py:26: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col, series in prediction.iteritems():
Capping the per_run_time_limit to 49.0 to have time for a least 2 models in each process.


2023-09-22 07:36:35,951 - Client-EnsembleBuilder - INFO - <Future: pending, key: fit_and_return_ensemble-28d7d180066cb9a2ae18a2d76a5c0fea>/<Client: 'inproc://172.30.0.2/51/1' processes=8 threads=8> Started Ensemble builder job at 2023.09.22-07.36.35 for iteration 0.
2023-09-22 07:36:36,037 - Client-EnsembleBuilder - DEBUG - Restricting your function to 3072 mb memory.
2023-09-22 07:36:36,084 - Client-EnsembleBuilder - DEBUG - Restricting your function to 65 seconds wall time.
2023-09-22 07:36:36,088 - Client-EnsembleBuilder - DEBUG - Allowing a grace period of 0 seconds.
2023-09-22 07:36:36,096 - Client-EnsembleBuilder - DEBUG - Function called with argument: (70.69709277153015, 0), {}


No models better than random - using Dummy losses!
	Models besides current dummy model: 0
	Dummy models: 1


2023-09-22 07:36:45,003 - Client-EnsembleBuilder - DEBUG - iteration=1 @ elapsed_time=36.496055603027344 has history=[{'Timestamp': Timestamp('2023-09-22 07:36:36.378413'), 'ensemble_optimization_score': 0.9276169265033407}]
2023-09-22 07:36:45,032 - Client-EnsembleBuilder - INFO - <Future: pending, key: fit_and_return_ensemble-25c974729281632db03927eba41993a3>/<Client: 'inproc://172.30.0.2/51/1' processes=8 threads=8> Started Ensemble builder job at 2023.09.22-07.36.45 for iteration 1.
2023-09-22 07:36:45,160 - Client-EnsembleBuilder - DEBUG - Restricting your function to 3072 mb memory.
2023-09-22 07:36:45,179 - Client-EnsembleBuilder - DEBUG - Restricting your function to 56 seconds wall time.
2023-09-22 07:36:45,187 - Client-EnsembleBuilder - DEBUG - Allowing a grace period of 0 seconds.
2023-09-22 07:36:45,197 - Client-EnsembleBuilder - DEBUG - Function called with argument: (61.52154803276062, 1), {}


No models better than random - using Dummy losses!
	Models besides current dummy model: 0
	Dummy models: 1


...

2023-09-22 07:37:30,415 - Client-EnsembleBuilder - DEBUG - iteration=9 @ elapsed_time=81.92051696777344 has history=[]
2023-09-22 07:37:30,427 - Client-EnsembleBuilder - INFO - <Future: pending, key: fit_and_return_ensemble-668e0471875ff3697caa1634dd863c48>/<Client: 'inproc://172.30.0.2/51/1' processes=8 threads=8> Started Ensemble builder job at 2023.09.22-07.37.30 for iteration 9.
2023-09-22 07:37:30,522 - Client-EnsembleBuilder - DEBUG - Restricting your function to 3072 mb memory.
2023-09-22 07:37:30,524 - Client-EnsembleBuilder - DEBUG - Restricting your function to 11 seconds wall time.
2023-09-22 07:37:30,526 - Client-EnsembleBuilder - DEBUG - Allowing a grace period of 0 seconds.
2023-09-22 07:37:30,547 - Client-EnsembleBuilder - DEBUG - Function called with argument: (16.15927815437317, 9), {}


No models better than random - using Dummy losses!
	Models besides current dummy model: 0
	Dummy models: 1


2023-09-22 07:37:35,156 - Client-EnsembleBuilder - DEBUG - iteration=10 @ elapsed_time=86.65148282051086 has history=[]
2023-09-22 07:37:35,218 - Client-EnsembleBuilder - INFO - <Future: pending, key: fit_and_return_ensemble-de01f2c87d43eec1bf97e9ccab557d99>/<Client: 'inproc://172.30.0.2/51/1' processes=8 threads=8> Started Ensemble builder job at 2023.09.22-07.37.35 for iteration 10.
2023-09-22 07:37:35,409 - Client-EnsembleBuilder - DEBUG - Restricting your function to 3072 mb memory.
2023-09-22 07:37:35,442 - Client-EnsembleBuilder - DEBUG - Restricting your function to 6 seconds wall time.
2023-09-22 07:37:35,490 - Client-EnsembleBuilder - DEBUG - Allowing a grace period of 0 seconds.
2023-09-22 07:37:35,547 - Client-EnsembleBuilder - DEBUG - Function called with argument: (11.285644054412842, 10), {}


No models better than random - using Dummy losses!
	Models besides current dummy model: 0
	Dummy models: 1


AutoSklearn2Classifier(delete_tmp_folder_after_terminate=False,
                       logging_config={'disable_existing_loggers': True,
                                       'formatters': {'custom': {'format': '%(asctime)s '
                                                                           '- '
                                                                           '%(name)s '
                                                                           '- '
                                                                           '%(levelname)s '
                                                                           '- '
                                                                           '%(message)s'}},
                                       'handlers': {'console': {'class': 'logging.StreamHandler',
                                                                'formatter': 'custom',
                                                                'level': 'DEB

In [17]:
predictions = automl.predict(X_test)
print("Accuracy score:", accuracy_score(y_test, predictions))

Accuracy score: 0.9276169265033407


## Auto-Sklearn1

In [18]:
from autosklearn.classification import AutoSklearnClassifier
from autosklearn import metrics
import pandas as pd
import logging
import wandb
import numpy as np

ModuleNotFoundError: No module named 'wandb'

In [3]:
#wandb.login()

In [4]:
# start a new wandb run to track this script
#wandb.init(
#    # set the wandb project where this run will be logged
#    project="automl",
#    entity='dagstuhl-23372'
#)

In [5]:
from WandbLogger import WandbHandler

In [6]:
# Define your logging configuration
logging_config = {
    "version": 1,
    "disable_existing_loggers": True,
    "formatters": {
        "custom": {
            "format": "%(asctime)s - %(name)s - %(levelname)s - %(message)s"
        }
    },
    "handlers": {
        "console": {
            "level": "DEBUG",
            "formatter": "custom",
            "class": "logging.StreamHandler",
            "stream": "ext://sys.stdout",
        },
        "wandb": {
            "level": "DEBUG",  # Adjust the logging level as needed
            "formatter": "custom",
            "class": "WandbLogger.WandbHandler", 
        },
    },
    "loggers": {
        "": {
            "level": "DEBUG",
            "handlers": ["console", "wandb"],  # Use both console and WandB handlers
        },
        "Client-EnsembleBuilder": {
            "level": "DEBUG",
            "handlers": ["console", "wandb"],  # Use both console and WandB handlers
        },
    },
}


# Configure the logging with the provided configuration
logging.config.dictConfig(logging_config)

# Now you can use the logging module as usual, and log messages will be sent to WandB
logging.info("This is an INFO-level log message.")


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ create logger
2023-09-20 07:49:30,270 - root - INFO - This is an INFO-level log message.


In [7]:
# link from GDrive sharing dialogue
test_url='https://drive.google.com/file/d/11MQS4q8DGpD1-jk2sqJ1lJKVmpC35ECF/view?usp=sharing'
# create the actual linnk
test_url='https://drive.google.com/uc?id=' + test_url.split('/')[-2]
df_test = pd.read_csv('preprocessed-test.csv')
df_test = df_test.loc[:, ~df_test.columns.str.contains('^Unnamed')]
df_test = df_test.replace(['inactive', 'active'], [0,1]) # swap -1 (inactive) and 0 (unknown)

train_url='https://drive.google.com/file/d/1nDuqeKMsScBJZxplxH7Chxv1JLbKJARX/view?usp=drive_link'
train_url='https://drive.google.com/uc?id=' + train_url.split('/')[-2]
df_train = pd.read_csv('preprocessed-train.csv')
df_train = df_train.loc[:, ~df_train.columns.str.contains('^Unnamed')]
df_train = df_train.replace(['inactive', 'active'], [0,1]) # swap -1 (inactive) and 0 (unknown)
df_train

,label,0,1,2,3,4,5,6,7,8,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3587,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3588,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3589,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3590,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [8]:
y = df_train['label']
X = df_train.loc[:, df_train.columns != "label"]
X.to_numpy()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    train_size=0.75, test_size=0.25)

In [10]:
def wandb_accuracy(solution, prediction):
    accuracy = metrics.accuracy(solution, prediction)
    logging.debug({"custom_log": f"accuracy={accuracy}"})
    return accuracy
    

my_scorer = metrics.accuracy;
my_scorer.score_func=wandb_accuracy

my_scorer = metrics.make_scorer(
    name="wandb_accu",
    score_func=wandb_accuracy,
    optimum=1,
    greater_is_better=True,
    needs_proba=False,
    needs_threshold=False,
)

In [11]:
logging.info({"custom_log": "870 2023-09-14 12:21:14,601 - Client-TAE - INFO - Starting to evaluate configuration 9 "})
wandb_accuracy(np.array([0]),np.array([1]))
wandb_accuracy(np.array([0,1]),np.array([1,1]))
logging.info({"custom_log": "909 2023-09-14 12:21:19,988 - Client-TAE - INFO - Finished evaluating configuration 9"})

2023-09-20 07:49:30,909 - root - INFO - {'custom_log': '870 2023-09-14 12:21:14,601 - Client-TAE - INFO - Starting to evaluate configuration 9 '}
2023-09-20 07:49:30,914 - root - DEBUG - {'custom_log': 'accuracy=0.0'}
2023-09-20 07:49:30,918 - root - DEBUG - {'custom_log': 'accuracy=0.5'}
2023-09-20 07:49:30,920 - root - INFO - {'custom_log': '909 2023-09-14 12:21:19,988 - Client-TAE - INFO - Finished evaluating configuration 9'}


In [12]:
automl = AutoSklearnClassifier(
    # Pass the config file we created
    logging_config=logging_config,
    metric=my_scorer,
    time_left_for_this_task=100
)
automl.fit(X_train, y_train)

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ create logger
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ create logger
2023-09-20 07:49:43,214 - Client-autosklearn.automl_common.common.utils.backend - DEBUG - _make_internals_directory: [Errno 17] File exists: '/tmp/auto-sklearn_tmp_3b741c31-578a-11ee-840a-0242ac110002/.auto-sklearn'
2023-09-20 07:49:43,215 - Client-autosklearn.automl_common.common.utils.backend - DEBUG - _make_internals_directory: [Errno 17] File exists: '/tmp/auto-sklearn_tmp_3b741c31-578a-11ee-840a-0242ac110002/.auto-sklearn/runs'
2023-09-20 07:49:43,540 - Client-AutoML(1):3b7484e2-578a-11ee-840a-0242ac110002 - DEBUG - Starting to print environment information
2023-09-20 07:49:43,541 - Client-AutoML(1):3b7484e2-578a-11ee-840a-0242ac110002 - DEBUG -   Python version: ['3.9.13 | packaged by conda-forge | (main, May 27 2022, 16:56:21) ', '[GCC 10.3.0]']
2023-09-20 07:49:43,563 - Client-AutoML(1):3b7484e2-578a-11ee-840a-0242ac110002 - DEBUG - 	Distributi

AutoSklearnClassifier(ensemble_class=<class 'autosklearn.ensembles.ensemble_selection.EnsembleSelection'>,
                      logging_config={'disable_existing_loggers': True,
                                      'formatters': {'custom': {'format': '%(asctime)s '
                                                                          '- '
                                                                          '%(name)s '
                                                                          '- '
                                                                          '%(levelname)s '
                                                                          '- '
                                                                          '%(message)s'}},
                                      'handlers': {'console': {'class': 'logging.StreamHandler',
                                                               'formatter': 'custom',
                                            

In [16]:
# View the models found by auto-sklearn
print(automl.leaderboard()) 

          rank  ensemble_weight           type      cost  duration
model_id                                                          
2            1              1.0  random_forest  0.067416  5.699415


In [17]:
from pprint import pprint
# Print the final ensemble constructed by auto-sklearn
pprint(automl.show_models(), indent=4)

{   2: {   'balancing': Balancing(random_state=1),
           'classifier': <autosklearn.pipeline.components.classification.ClassifierChoice object at 0x7f36908568e0>,
           'cost': 0.0674157303370787,
           'data_preprocessor': <autosklearn.pipeline.components.data_preprocessing.DataPreprocessorChoice object at 0x7f3690764670>,
           'ensemble_weight': 1.0,
           'feature_preprocessor': <autosklearn.pipeline.components.feature_preprocessing.FeaturePreprocessorChoice object at 0x7f3690856be0>,
           'model_id': 2,
           'rank': 1,
           'sklearn_classifier': RandomForestClassifier(max_features=29, n_estimators=512, n_jobs=1,
                       random_state=1, warm_start=True)}}


In [18]:
from sklearn.metrics import accuracy_score
predictions = automl.predict(X_test)
print("Accuracy score:", accuracy_score(y_test, predictions))

Accuracy score: 0.9276169265033407
